# Refresh Power BI Semantic Model via REST endpoint

Power BI allows the refresh of a published model via the executeQueries endpoint.

[Datasets - Refresh Dataset In Group](https://learn.microsoft.com/en-us/rest/api/power-bi/datasets/refresh-dataset-in-group?wt.mc_id=MVP_449122&source=post_page-----1e52165e36ab---------------------------------------)

Authetication can be done via SPN (Microsoft Entra Application).

# Prerequisites

To trigger the refresh, we first need to collect the following information:
- The tenant ID that hosts the semantic model
- The client ID of the SPN object
- The secret of the SPN object
- The semantic model ID
- The workspace ID

# Get Oauth2 Token for the SPN

We can request for a token for our SPN leveraging **sp_invoke_external_rest_endpoint** and we save it an @access_token variable.

In [ ]:

DECLARE @tenant_id NVARCHAR(100) = N'<redacted>'
    ,@client_id NVARCHAR(100) = N'<redacted>'
    ,@client_secret NVARCHAR(200) = N'<redacted>'
    ,@workspace_id NVARCHAR(100) = N'<redacted>'
    ,@dataset_id NVARCHAR(100) = N'<redacted>'

DECLARE @scope NVARCHAR(200) = 
  N'https://analysis.windows.net/powerbi/api/.default';

DECLARE @response NVARCHAR(MAX);

DECLARE @url NVARCHAR(400) = 
  N'https://login.microsoftonline.com/' 
    + @tenant_id 
    + N'/oauth2/v2.0/token';

DECLARE @headers NVARCHAR(MAX) = 
  N'{"Content-Type":"application/x-www-form-urlencoded"}';

DECLARE @body NVARCHAR(MAX) =
  N'client_id=' + @client_id 
  + N'&scope=' + @scope 
  + N'&client_secret=' + @client_secret 
  + N'&grant_type=client_credentials';
  
EXEC sp_invoke_external_rest_endpoint
  @method = 'POST'
  ,@url = @url
  ,@headers = @headers
  ,@payload = @body
  ,@credential = NULL
  ,@response = @response OUTPUT;
  
DECLARE @access_token NVARCHAR(MAX) = 
  JSON_VALUE(@response, N'$.result.access_token');

# Refresh the semantic model

Once you have the token stored in the @access_token variable, you can use it to call the Power BI Refresh API.

In [ ]:
DECLARE @refreshUrl NVARCHAR(400) =
  N'https://api.powerbi.com/v1.0/myorg/groups/' 
    + @workspace_id 
    + N'/datasets/' 
    + @dataset_id 
    + N'/refreshes';

DECLARE @refreshHeaders NVARCHAR(MAX) =
    N'{"Authorization":"Bearer ' 
      + @access_token 
      + N'","Content-Type":"application/json"}';

DECLARE @refreshBody NVARCHAR(MAX) = N'{"notifyOption": "NoNotification"}';

DECLARE @refreshResponse NVARCHAR(MAX);

EXEC sp_invoke_external_rest_endpoint
    @method = 'POST'
    ,@url = @refreshUrl
    ,@headers = @refreshHeaders
    ,@payload = @refreshBody
    ,@credential = NULL
    ,@response = @refreshResponse OUTPUT;    

SELECT @refreshResponse;